In [1]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import imutils
import time
import cv2
import pytesseract
#from vidstab import VidStab

In [2]:
def decode_predictions(scores, geometry):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    for y in range(0, numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        for x in range(0, numCols):
            if scoresData[x] < min_confidence:
                continue

            (offsetX, offsetY) = (x * 4.0, y * 4.0)
#             print("x=",x, "y=", y)
#             print("offsetX,offsetY",(offsetX, offsetY))

            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            
#             print("cos=", cos, "sin=",sin)

            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
#             print("h=", h, "w=",w)

            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
#             print("xData=",xData0[x],xData1[x],xData2[x],xData3[x],"endX=",endX,"endY=",endY,"startX=",startX,"startY=",startY,"angle=",angle)

    return (rects, confidences)

In [3]:
east = 'frozen_east_text_detection.pb'
video =''
min_confidence = 0.5
width = 350
height = 350

In [4]:
# initialize the original frame dimensions, new frame dimensions,
# and ratio between the dimensions
(W, H) = (None, None)
(newW, newH) = (width, height)
(rW, rH) = (None, None)

# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(east)

# if a video path was not supplied, grab the reference to the web cam
if video == '':
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    time.sleep(1.0)

# otherwise, grab a reference to the video file
else:
    vs = cv2.VideoCapture(video)

# start the FPS throughput estimator
fps = FPS().start()

# loop over frames from the video stream
while True:
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
    frame = vs.read()
    frame = frame[1] if "video"=='' else frame

    # check to see if we have reached the end of the stream
    if frame is None:
        break

    # resize the frame, maintaining the aspect ratio
    frame = imutils.resize(frame, width=1000)
    orig = frame.copy()

    # if our frame dimensions are None, we still need to compute the
    # ratio of old frame dimensions to new frame dimensions
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        rW = W / float(newW)
        rH = H / float(newH)

    # resize the frame, this time ignoring aspect ratio
    frame = cv2.resize(frame, (newW, newH))

    # construct a blob from the frame and then perform a forward pass
    # of the model to obtain the two output layer sets
    blob = cv2.dnn.blobFromImage(frame, 1.0, (newW, newH),(123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)

    # decode the predictions, then  apply non-maxima suppression to
    # suppress weak, overlapping bounding boxes
    (rects, confidences) = decode_predictions(scores, geometry)
    for index in range(len(rects)):
        (startX, startY, endX, endY)=rects[index]
        if startX < 0:
            startX=0
        rects[index]=(startX, startY, endX, endY)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    
    results = []

    for (startX, startY, endX, endY) in boxes:

        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)

                
        roi = orig[startY:endY, startX:endX]
        config = ("-l eng --oem 3 --psm 11")
        text = pytesseract.image_to_string(roi, config=config)
                       
        results.append(((startX, startY, endX, endY), text))
        
        
    # sort the results bounding box coordinates from top to bottom
    results = sorted(results, key=lambda r:r[0][1])
    output = orig.copy()
    
    for ((startX, startY, endX, endY), text) in results:
        print("OCR TEXT")
        print("========")
        print("{}\n".format(text))

        text = "".join([c if ord(c) < 128 else "" for c in text]).strip()

        cv2.rectangle(output, (startX, startY), (endX, endY),(0, 255, 0), 2)
        cv2.putText(output, text, (startX, startY - 20),cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

#     directory = 'C:\\Users\\ckunwar\\Test_Images\\'
#     with open(directory+"\\"+output+".txt",'w') as f: f.write(str(text))
    # show the output image
    cv2.imshow("Text Detection", output)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    #success,image = capture.read()
    
cv2.destroyAllWindows()


[INFO] loading EAST text detector...
[INFO] starting video stream...
